In [1]:
import numpy as np
import mdtraj as md 
import deeptime
from deeptime.decomposition import TICA
from itertools import combinations
import matplotlib.pyplot as plt

from glob import glob

In [2]:

pos_path = '/local_scratch2/jacopo/trans_temp/all_atom_replica_results_chignolin/exchange_all/coords_nowater/temperature_301'

In [5]:
dihedral_atoms = [(i, i+1, i+2, i+3) for i in range(7)]
angle_atoms = [(i, i+1, i+2) for i in range(8)]
print(f'Dihedral atoms: {dihedral_atoms}')

distances = []
dihedrals = []
angles = []
coords = []


coord_fns = sorted(glob(f'{pos_path}/coor*'))
#assert len(coord_fns) == 3744

# load topology and consider CA resolution
aa_pdb = md.load("./chi_sys.pdb").remove_solvent()
ca_idx = aa_pdb.topology.select("name CA")
print(ca_idx)

assert len(ca_idx) == 10

for fn in coord_fns:
    coords = np.load(fn) / 10.0
    print(coords.shape)
    aa_pdb.xyz = coords
    aa_pdb.time = np.arange(len(coords))
    cg_pdb = aa_pdb.atom_slice(ca_idx)
    ca_atoms = cg_pdb.topology.select("name CA")
    ca_atom_pairs = list(combinations(ca_atoms, 2))

    distances = md.compute_distances(cg_pdb, ca_atom_pairs)
    dihedrals = md.compute_dihedrals(cg_pdb, dihedral_atoms)
    angles = md.compute_angles(cg_pdb, angle_atoms)
    distances.append(distances)
    dihedrals.append(dihedrals)
    angles.append(angles)

all_distances = np.concatenate(distances)
all_dihedrals = np.concatenate(dihedrals)
all_angles = np.concatenate(angles)
print(f'Shape of all distances: {all_distances.shape}')

Dihedral atoms: [(0, 1, 2, 3), (1, 2, 3, 4), (2, 3, 4, 5), (3, 4, 5, 6), (4, 5, 6, 7), (5, 6, 7, 8), (6, 7, 8, 9)]
[  8  29  50  70  76  91 105 112 126 150]
(495, 175, 3)


ValueError: unitcell_lengths must be shape (495, 3). You supplied  (1, 3)

In [11]:
#aa_pdb.atom_slice(aa_pdb.topology.select("name CA"))
aa_pdb.unitcell_lengths

array([[5.9921, 5.9921, 5.9921]], dtype=float32)

In [ ]:
# determine converged TICA
tic_objs = []
lags =[1,20,40,60,120,160]
for lag in lags:
    tica = TICA(lagtime=lag, dim=2, scaling='kinetic_map')
    #tica_model = tica.fit(distances).fetch_model()
    tic_objs.append(tica.fit(distances).fetch_model())


# plot involved timescales
ts = np.array([obj.timescales for obj in tic_objs])
print(ts.shape)
for t in range(ts.shape[1]):
    plt.plot(lags,ts[:,t])
plt.yscale('log')
plt.show()


In [ ]:
# Plot ref TICA
chosen_tica = TICA(lagtime=60, dim=2, scaling='kinetic_map')
chosen_tica_model = chosen_tica.fit(distances).fetch_model()

tica_projection = chosen_tica_model.transform(distances)


#fig, axes = plt.subplots(figsize=(10, 8))

energy_landscape = deeptime.util.energy2d(tica_projection[:,0], tica_projection[:,1], kbt=1.0)
deeptime.plots.plot_energy2d(energy_landscape)

plt.ylabel('TIC 2')
plt.xlabel('TIC 1')

plt.show()
#deeptime.plots.plot_feature_histograms(tica_projection[:,:2],feature_labels=['TIC 1', 'TIC 2'], ax=axes[1,0])

#for ax in axes.flat[:2]:
#    ax.set_xlabel('TIC 1')
#    ax.set_ylabel('TIC 2')
#
#axes[0,1].set_xlim(axes[0,0].get_xlim())
#axes[0,1].set_ylim(axes[0,0].get_ylim())
#axes[1,1].set_xlim(axes[1,0].get_xlim())
#axes[1,1].set_ylim(axes[1,0].get_ylim())
#
#
#axes[0,0].set_title('Reference')
#axes[0,1].set_title(f'Simulated model {code}')
#fig.tight_layout()